# Library

In [26]:
import random
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import itertools
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.metrics import mean_absolute_error
import numpy as np
import json
import xgboost as xgb
import joblib

### 건고추

In [27]:
train = pd.read_csv("train_건고추.csv", encoding = 'cp949')
test = pd.read_csv("test_건고추.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

y_preds = {}

best_params_dict = {
    "1순": {
        9377: {"n_estimators": 1923, "max_depth": 19, "learning_rate": 0.04652056754648054, "subsample": 0.5334319533061308, "colsample_bytree": 0.7157206794031474, "min_child_weight": 5, "reg_alpha": 5.999507031063397, "reg_lambda": 5.965810644325051},
        9555: {"n_estimators": 2546, "max_depth": 13, "learning_rate": 0.16579092745704765, "subsample": 0.6060299182480591, "colsample_bytree": 0.9708200176228823, "min_child_weight": 1, "reg_alpha": 7.4250907487446165, "reg_lambda": 6.411797891005662},
        3949: {"n_estimators": 2341, "max_depth": 10, "learning_rate": 0.022680961074002054, "subsample": 0.5384571948814648, "colsample_bytree": 0.7737512880905816, "min_child_weight": 9, "reg_alpha": 2.6358204575230264, "reg_lambda": 4.036711193081059},
        7924: {"n_estimators": 1723, "max_depth": 8, "learning_rate": 0.12430031703069766, "subsample": 0.6630419690672185, "colsample_bytree": 0.860855850902466, "min_child_weight": 8, "reg_alpha": 2.397803079696025, "reg_lambda": 2.3096165027927045},
    },
    "2순": {
        9377: {"n_estimators": 1809, "max_depth": 6, "learning_rate": 0.04843469890626298, "subsample": 0.5612377940240227, "colsample_bytree": 0.8700271060922277, "min_child_weight": 4, "reg_alpha": 5.14769116516129, "reg_lambda": 7.865079407548869},
        9555: {"n_estimators": 977, "max_depth": 10, "learning_rate": 0.1916263527189192, "subsample": 0.5520315489117554, "colsample_bytree": 0.633763546218434, "min_child_weight": 5, "reg_alpha": 3.2261172830399603, "reg_lambda": 7.768135610059157},
        3949: {"n_estimators": 2126, "max_depth": 19, "learning_rate": 0.04005417715343745, "subsample": 0.6156910760775579, "colsample_bytree": 0.6275728645386054, "min_child_weight": 9, "reg_alpha": 5.541760817906737, "reg_lambda": 2.735776263745732},
        7924: {"n_estimators": 1197, "max_depth": 5, "learning_rate": 0.15089399568643513, "subsample": 0.5952454515515543, "colsample_bytree": 0.5882566794259629, "min_child_weight": 9, "reg_alpha": 2.663564866577076, "reg_lambda": 8.898677117340878},
    },
    "3순": {
        9377: {"n_estimators": 1279, "max_depth": 18, "learning_rate": 0.028830971687850357, "subsample": 0.6150715564697662, "colsample_bytree": 0.7816280505795264, "min_child_weight": 9, "reg_alpha": 1.6113012819014827, "reg_lambda": 2.668149104059637},
        9555: {"n_estimators": 2532, "max_depth": 14, "learning_rate": 0.03607335088921562, "subsample": 0.5824653801518888, "colsample_bytree": 0.7994678273724702, "min_child_weight": 9, "reg_alpha": 8.270082012464316, "reg_lambda": 1.2391440633981459},
        3949: {"n_estimators": 1928, "max_depth": 13, "learning_rate": 0.025207055174597884, "subsample": 0.5534059386683795, "colsample_bytree": 0.5336074646555482, "min_child_weight": 6, "reg_alpha": 8.638214108020266, "reg_lambda": 6.543832735091604},
        7924: {"n_estimators": 995, "max_depth": 3, "learning_rate": 0.01737228061408959, "subsample": 0.5624562462155033, "colsample_bytree": 0.7975055224153474, "min_child_weight": 1, "reg_alpha": 3.1394040119829203, "reg_lambda": 0.14506398560359113},
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
        best_model.fit(X, y)

        model_save_path = f'건고추_XGB_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)

        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  
sub_test = pd.read_csv('Processte.csv',encoding='cp949')
sub_test.loc[sub_test['품목명'] == '건고추', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '건고추', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '건고추', '3순'] = y_preds['y_pred_3']


### 감자

In [28]:
train = pd.read_csv("train_감자.csv", encoding = 'cp949')
test = pd.read_csv("test_감자.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

y_preds = {}

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 2341, "max_depth": 11, "learning_rate": 0.015412701830521625, "subsample": 0.5859097714830525, "colsample_bytree": 0.5450411424306277, "min_child_weight": 4, "reg_alpha": 1.7831294525231889, "reg_lambda": 3.44528362180448},
        9555: {"n_estimators": 2435, "max_depth": 6, "learning_rate": 0.14327606903054724, "subsample": 0.6163964916669449, "colsample_bytree": 0.5129549776200466, "min_child_weight": 4, "reg_alpha": 6.437426086117856, "reg_lambda": 5.211573870815037},
        3949: {"n_estimators": 2188, "max_depth": 18, "learning_rate": 0.04891694069922088, "subsample": 0.680412174212124, "colsample_bytree": 0.5797986501138502, "min_child_weight": 4, "reg_alpha": 1.8800491126838252, "reg_lambda": 4.086341344333814},
        7924: {"n_estimators": 433, "max_depth": 9, "learning_rate": 0.08009410260180493, "subsample": 0.7453162717856692, "colsample_bytree": 0.5953183570158112, "min_child_weight": 4, "reg_alpha": 0.6781037756243042, "reg_lambda": 8.040011147111919},
    },
    "2순": {
        9377: {"n_estimators": 1074, "max_depth": 3, "learning_rate": 0.24189314990946606, "subsample": 0.5940256820633358, "colsample_bytree": 0.6343296343701532, "min_child_weight": 8, "reg_alpha": 4.757560293408634, "reg_lambda": 2.062101037942351},
        9555: {"n_estimators": 1699, "max_depth": 14, "learning_rate": 0.1318465912590907, "subsample": 0.5928778560004319, "colsample_bytree": 0.8557171077339198, "min_child_weight": 5, "reg_alpha": 7.0035983676229625, "reg_lambda": 2.398035642356394},
        3949: {"n_estimators": 1030, "max_depth": 10, "learning_rate": 0.10321813569156357, "subsample": 0.5192632867149305, "colsample_bytree": 0.6295281874227744, "min_child_weight": 5, "reg_alpha": 1.230366752864514, "reg_lambda": 1.6405771333127226},
        7924: {"n_estimators": 869, "max_depth": 6, "learning_rate": 0.15211518999436222, "subsample": 0.500684210332173, "colsample_bytree": 0.9448913835460466, "min_child_weight": 10, "reg_alpha": 4.5914417256233495, "reg_lambda": 7.507991568313692},
    },
    "3순": {
        9377: {"n_estimators": 663, "max_depth": 6, "learning_rate": 0.22822838060801376, "subsample": 0.5036521159573091, "colsample_bytree": 0.8703158177976755, "min_child_weight": 6, "reg_alpha": 9.45346123190003, "reg_lambda": 5.014642893567626},
        9555: {"n_estimators": 1387, "max_depth": 8, "learning_rate": 0.03289525685673032, "subsample": 0.5177413757422344, "colsample_bytree": 0.8091898188559641, "min_child_weight": 4, "reg_alpha": 8.523681852324415, "reg_lambda": 0.0027756839939803157},
        3949: {"n_estimators": 2338, "max_depth": 11, "learning_rate": 0.08745195664484588, "subsample": 0.5555575221066396, "colsample_bytree": 0.7874695709390427, "min_child_weight": 9, "reg_alpha": 7.964892526922062, "reg_lambda": 6.768695802547276},
        7924: {"n_estimators": 474, "max_depth": 4, "learning_rate": 0.21495309680047114, "subsample": 0.5508877130143965, "colsample_bytree": 0.5666842066295218, "min_child_weight": 1, "reg_alpha": 5.036757276613314, "reg_lambda": 1.0528339616232207},
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4): 
    seed_test_preds = []
    순_key = f'{i}순' 

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
        best_model.fit(X, y)

        model_save_path = f'감자_XGB_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '감자', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '감자', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '감자', '3순'] = y_preds['y_pred_3']


### 깐마늘(국산)

In [29]:
train = pd.read_csv("train_깐마늘_국산.csv", encoding = 'cp949')
test = pd.read_csv("test_깐마늘_국산.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

y_preds = {}

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 2657, "max_depth": 8, "learning_rate": 0.2997897457912238, "subsample": 0.9026869352575697, "colsample_bytree": 0.893460874411241, "min_child_weight": 8, "reg_alpha": 2.1621511221459473, "reg_lambda": 6.379338035184857},
        9555: {"n_estimators": 1951, "max_depth": 6, "learning_rate": 0.2981100413293301, "subsample": 0.8973839591958276, "colsample_bytree": 0.7976569178022983, "min_child_weight": 8, "reg_alpha": 1.8199443922923473, "reg_lambda": 2.8242001018446605},
        3949: {"n_estimators": 2460, "max_depth": 20, "learning_rate": 0.2904038014880782, "subsample": 0.9307170092998699, "colsample_bytree": 0.832473945932362, "min_child_weight": 7, "reg_alpha": 0.2502500343734919, "reg_lambda": 2.9535121948225864},
        7924: {"n_estimators": 1626, "max_depth": 19, "learning_rate": 0.28321215330234695, "subsample": 0.9186314025100732, "colsample_bytree": 0.9806496766023839, "min_child_weight": 8, "reg_alpha": 3.4674147420767367, "reg_lambda": 0.5697034748018113},
    },
    "2순": {
        9377: {"n_estimators": 2895, "max_depth": 14, "learning_rate": 0.22352310990415156, "subsample": 0.703494342753191, "colsample_bytree": 0.6626992194830694, "min_child_weight": 8, "reg_alpha": 7.72498929052509, "reg_lambda": 9.26636560444955},
        9555: {"n_estimators": 2795, "max_depth": 4, "learning_rate": 0.29397707822423014, "subsample": 0.9810298944222359, "colsample_bytree": 0.7503128490672966, "min_child_weight": 9, "reg_alpha": 6.08912105878191, "reg_lambda": 0.4790295911954692},
        3949: {"n_estimators": 1271, "max_depth": 12, "learning_rate": 0.2510957263789993, "subsample": 0.7135642888895986, "colsample_bytree": 0.9583765796497234, "min_child_weight": 8, "reg_alpha": 3.2899939692840894, "reg_lambda": 2.926042414380865},
        7924: {"n_estimators": 2708, "max_depth": 19, "learning_rate": 0.27264296830254653, "subsample": 0.99540008744542, "colsample_bytree": 0.6933217861936607, "min_child_weight": 9, "reg_alpha": 0.08010880166947623, "reg_lambda": 7.054341954222285},
    },
    "3순": {
        9377: {"n_estimators": 2593, "max_depth": 17, "learning_rate": 0.18028850842383973, "subsample": 0.9622711286939722, "colsample_bytree": 0.6344803677470267, "min_child_weight": 10, "reg_alpha": 5.641933428845374, "reg_lambda": 6.098681981484663},
        9555: {"n_estimators": 2889, "max_depth": 6, "learning_rate": 0.24231391585386905, "subsample": 0.6673384583204696, "colsample_bytree": 0.5602996340797951, "min_child_weight": 10, "reg_alpha": 9.092219652343779, "reg_lambda": 1.082011184835265},
        3949: {"n_estimators": 2033, "max_depth": 16, "learning_rate": 0.1758925324532521, "subsample": 0.9289228134786575, "colsample_bytree": 0.9065544611450423, "min_child_weight": 10, "reg_alpha": 6.194687204428908, "reg_lambda": 2.4755300198669055},
        7924: {"n_estimators": 2669, "max_depth": 9, "learning_rate": 0.1819496346070102, "subsample": 0.7701958470218522, "colsample_bytree": 0.819956980885592, "min_child_weight": 10, "reg_alpha": 2.629680302905466, "reg_lambda": 8.10681237887405},
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4): 
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
        best_model.fit(X, y)

        model_save_path = f'깐마늘(국산)_XGB_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '깐마늘(국산)', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '깐마늘(국산)', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '깐마늘(국산)', '3순'] = y_preds['y_pred_3']


### 대파

In [30]:
train = pd.read_csv("train_대파.csv", encoding = 'cp949')
test = pd.read_csv("test_대파.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

y_preds = {}


best_params_dict= {
    "1순": {
        9377: {"n_estimators": 359, "max_depth": 7, "learning_rate": 0.0984640985652742, "subsample": 0.9755675108674087, "colsample_bytree": 0.5592124107775549, "min_child_weight": 2, "reg_alpha": 5.59946202347779, "reg_lambda": 5.358989453552983},
        9555: {"n_estimators": 2679, "max_depth": 15, "learning_rate": 0.02055795484460717, "subsample": 0.9858381593495669, "colsample_bytree": 0.5412300749816026, "min_child_weight": 1, "reg_alpha": 4.429510107979178, "reg_lambda": 4.859212939637951},
        3949: {"n_estimators": 1912, "max_depth": 10, "learning_rate": 0.12992356240891764, "subsample": 0.9196711446923945, "colsample_bytree": 0.5583526334490962, "min_child_weight": 1, "reg_alpha": 3.946226439375406, "reg_lambda": 7.585546868226279},
        7924: {"n_estimators": 629, "max_depth": 11, "learning_rate": 0.095268091708072, "subsample": 0.9347073270960663, "colsample_bytree": 0.7045464418261608, "min_child_weight": 2, "reg_alpha": 3.9722151928481857, "reg_lambda": 6.657145213135394},
    },
    "2순": {
        9377: {"n_estimators": 725, "max_depth": 6, "learning_rate": 0.27076156646613747, "subsample": 0.6894020723909529, "colsample_bytree": 0.6594717527623305, "min_child_weight": 3, "reg_alpha": 1.855241020830033, "reg_lambda": 2.635516891345371},
        9555: {"n_estimators": 1955, "max_depth": 20, "learning_rate": 0.293768485105688, "subsample": 0.8748490443525189, "colsample_bytree": 0.5028578779792372, "min_child_weight": 1, "reg_alpha": 3.199108837889445, "reg_lambda": 4.483896684917806},
        3949: {"n_estimators": 2721, "max_depth": 10, "learning_rate": 0.040716031930131416, "subsample": 0.8793247371917796, "colsample_bytree": 0.660972540059019, "min_child_weight": 5, "reg_alpha": 0.5710968826198275, "reg_lambda": 5.728695854408228},
        7924: {"n_estimators": 1817, "max_depth": 5, "learning_rate": 0.16568125262445038, "subsample": 0.724262252657609, "colsample_bytree": 0.7679353750625274, "min_child_weight": 3, "reg_alpha": 4.399570275718538, "reg_lambda": 2.401187083386903},
    },
    "3순": {
        9377: {"n_estimators": 1931, "max_depth": 11, "learning_rate": 0.18110078138193625, "subsample": 0.9628794703278177, "colsample_bytree": 0.615913805706384, "min_child_weight": 2, "reg_alpha": 2.242332585480615, "reg_lambda": 3.282493082940931},
        9555: {"n_estimators": 1505, "max_depth": 10, "learning_rate": 0.14609870420921559, "subsample": 0.957790129379699, "colsample_bytree": 0.5083929178080872, "min_child_weight": 4, "reg_alpha": 5.600905644412256, "reg_lambda": 7.810346636380197},
        3949: {"n_estimators": 1002, "max_depth": 19, "learning_rate": 0.07448926768508218, "subsample": 0.9732752056393886, "colsample_bytree": 0.6104230784178994, "min_child_weight": 3, "reg_alpha": 0.4511792986357669, "reg_lambda": 4.368719202758369},
        7924: {"n_estimators": 1288, "max_depth": 18, "learning_rate": 0.23577011967783892, "subsample": 0.9297326188079662, "colsample_bytree": 0.8669970878240895, "min_child_weight": 3, "reg_alpha": 0.04608970456515815, "reg_lambda": 8.093032074293628},
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4): 
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
        best_model.fit(X, y)

        model_save_path = f'대파_XGB_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '대파', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '대파', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '대파', '3순'] = y_preds['y_pred_3']


### 무

In [31]:
train = pd.read_csv("train_무.csv", encoding = 'cp949')
test = pd.read_csv("test_무.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

y_preds = {}

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 1511, "max_depth": 16, "learning_rate": 0.11581339385083773, "subsample": 0.5715886205863586, "colsample_bytree": 0.856593246335739, "min_child_weight": 2, "reg_alpha": 3.5819300458422334, "reg_lambda": 1.4691095942340482},
        9555: {"n_estimators": 2880, "max_depth": 6, "learning_rate": 0.019158368138887688, "subsample": 0.8412165820764654, "colsample_bytree": 0.8198771995465477, "min_child_weight": 3, "reg_alpha": 6.825253739709787, "reg_lambda": 0.07768160948474323},
        3949: {"n_estimators": 2595, "max_depth": 18, "learning_rate": 0.13209024754507617, "subsample": 0.7918695754405852, "colsample_bytree": 0.9203609468057712, "min_child_weight": 9, "reg_alpha": 7.616811284667843, "reg_lambda": 0.7084025918377677},
        7924: {"n_estimators": 706, "max_depth": 14, "learning_rate": 0.05966349486187719, "subsample": 0.975351161408665, "colsample_bytree": 0.9008062113501178, "min_child_weight": 4, "reg_alpha": 8.945375878067036, "reg_lambda": 0.00846143019957224},
    },
    "2순": {
        9377: {"n_estimators": 1353, "max_depth": 20, "learning_rate": 0.09812597675326684, "subsample": 0.7372898384714295, "colsample_bytree": 0.7499350770673643, "min_child_weight": 2, "reg_alpha": 8.815200094274285, "reg_lambda": 5.553629355229274},
        9555: {"n_estimators": 2672, "max_depth": 8, "learning_rate": 0.13137295852027087, "subsample": 0.8813335230388525, "colsample_bytree": 0.9127332825165446, "min_child_weight": 2, "reg_alpha": 3.505481360093038, "reg_lambda": 7.247361459294812},
        3949: {"n_estimators": 402, "max_depth": 8, "learning_rate": 0.14121954654783006, "subsample": 0.8899985811136056, "colsample_bytree": 0.5646805828900342, "min_child_weight": 2, "reg_alpha": 1.7691689652279006, "reg_lambda": 5.731498543065827},
        7924: {"n_estimators": 1776, "max_depth": 6, "learning_rate": 0.28772992764052346, "subsample": 0.6582504739136146, "colsample_bytree": 0.8682157162559964, "min_child_weight": 2, "reg_alpha": 6.451071758510017, "reg_lambda": 7.557315699004247},
    },
    "3순": {
        9377: {"n_estimators": 988, "max_depth": 7, "learning_rate": 0.24671207514611618, "subsample": 0.7819467441599348, "colsample_bytree": 0.9622535201532348, "min_child_weight": 3, "reg_alpha": 1.2408225716167047, "reg_lambda": 5.293089528001744},
        9555: {"n_estimators": 1695, "max_depth": 7, "learning_rate": 0.0884032869081284, "subsample": 0.7993036797653224, "colsample_bytree": 0.7788521315924819, "min_child_weight": 1, "reg_alpha": 1.7570921769919567, "reg_lambda": 7.977384894482953},
        3949: {"n_estimators": 2917, "max_depth": 15, "learning_rate": 0.2761443197468835, "subsample": 0.5856006103691742, "colsample_bytree": 0.8811633110686008, "min_child_weight": 1, "reg_alpha": 4.156430220361832, "reg_lambda": 9.630748586541205},
        7924: {"n_estimators": 1727, "max_depth": 15, "learning_rate": 0.12945922970370186, "subsample": 0.8403921517122799, "colsample_bytree": 0.8114709170274299, "min_child_weight": 1, "reg_alpha": 3.0743444445237187, "reg_lambda": 7.567639069977547},
    }
}


seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
        best_model.fit(X, y)

        model_save_path = f'무_XGB_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '무', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '무', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '무', '3순'] = y_preds['y_pred_3']


### 배추

In [32]:
train = pd.read_csv("train_배추.csv", encoding = 'cp949')
test = pd.read_csv("test_배추.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)


train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

y_preds = {}

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 731, "max_depth": 3, "learning_rate": 0.15728326993309352, "subsample": 0.948836995649512, "colsample_bytree": 0.9095623305306912, "min_child_weight": 8, "reg_alpha": 2.6440464023898818, "reg_lambda": 5.365005645106437},
        9555: {"n_estimators": 2949, "max_depth": 17, "learning_rate": 0.2228742055092585, "subsample": 0.5799503419597875, "colsample_bytree": 0.8496650472048766, "min_child_weight": 3, "reg_alpha": 7.991226240749917, "reg_lambda": 9.984857128995912},
        3949: {"n_estimators": 945, "max_depth": 17, "learning_rate": 0.17059069735106883, "subsample": 0.7468251788642863, "colsample_bytree": 0.5471407617948116, "min_child_weight": 2, "reg_alpha": 1.8173569760890849, "reg_lambda": 6.069241286746007},
        7924: {"n_estimators": 2378, "max_depth": 18, "learning_rate": 0.08356277531344279, "subsample": 0.6677377371154416, "colsample_bytree": 0.7240255919877567, "min_child_weight": 7, "reg_alpha": 4.750976081963245, "reg_lambda": 1.439597440196262},
    },
    "2순": {
        9377: {"n_estimators": 1106, "max_depth": 6, "learning_rate": 0.024648055752174054, "subsample": 0.5239075797244143, "colsample_bytree": 0.7061033729177608, "min_child_weight": 6, "reg_alpha": 1.0069318164148569, "reg_lambda": 8.49232777936879},
        9555: {"n_estimators": 477, "max_depth": 19, "learning_rate": 0.06782533643641112, "subsample": 0.6138320287567588, "colsample_bytree": 0.801184598264242, "min_child_weight": 6, "reg_alpha": 8.771704538477131, "reg_lambda": 2.907349816176465},
        3949: {"n_estimators": 2821, "max_depth": 17, "learning_rate": 0.062481796150389665, "subsample": 0.5004821687507287, "colsample_bytree": 0.5020874322858997, "min_child_weight": 5, "reg_alpha": 9.918047489424666, "reg_lambda": 4.836797095301563},
        7924: {"n_estimators": 474, "max_depth": 15, "learning_rate": 0.2285295155358468, "subsample": 0.5628102163062088, "colsample_bytree": 0.669281635263394, "min_child_weight": 3, "reg_alpha": 2.382116957968482, "reg_lambda": 3.058553173628421},
    },
    "3순": {
        9377: {"n_estimators": 2826, "max_depth": 11, "learning_rate": 0.0995898218588722, "subsample": 0.5727780425821573, "colsample_bytree": 0.7120382065712848, "min_child_weight": 7, "reg_alpha": 5.2999354422924885, "reg_lambda": 1.606767114892411},
        9555: {"n_estimators": 1231, "max_depth": 3, "learning_rate": 0.1332269978037241, "subsample": 0.5672284834608808, "colsample_bytree": 0.5362666149931072, "min_child_weight": 1, "reg_alpha": 2.8387267325224914, "reg_lambda": 2.3113526276724343},
        3949: {"n_estimators": 1206, "max_depth": 16, "learning_rate": 0.12966988485921985, "subsample": 0.5607106863186356, "colsample_bytree": 0.6199316467435861, "min_child_weight": 3, "reg_alpha": 8.642039667347802, "reg_lambda": 0.43251566794530694},
        7924: {"n_estimators": 1324, "max_depth": 5, "learning_rate": 0.19119983668694765, "subsample": 0.8267163943795055, "colsample_bytree": 0.9356086852526486, "min_child_weight": 2, "reg_alpha": 9.427465690313017, "reg_lambda": 1.1761935768160439},
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
        best_model.fit(X, y)

        model_save_path = f'배추_XGB_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '배추', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '배추', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '배추', '3순'] = y_preds['y_pred_3']


### 사과

In [33]:
train = pd.read_csv("train_사과.csv", encoding = 'cp949')
test = pd.read_csv("test_사과.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

y_preds = {}

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 2519, "max_depth": 18, "learning_rate": 0.010112813865660987, "subsample": 0.5333096346103383, "colsample_bytree": 0.5960086316627993, "min_child_weight": 10, "reg_alpha": 6.733205033610161, "reg_lambda": 1.3297498700929364},
        9555: {"n_estimators": 2596, "max_depth": 7, "learning_rate": 0.2565471120429437, "subsample": 0.7882596294202371, "colsample_bytree": 0.6644900578714683, "min_child_weight": 10, "reg_alpha": 1.0178283883835504, "reg_lambda": 5.332629227250732},
        3949: {"n_estimators": 1310, "max_depth": 12, "learning_rate": 0.17387795541656836, "subsample": 0.8796376165761329, "colsample_bytree": 0.5845725752046038, "min_child_weight": 10, "reg_alpha": 2.3477142235267237, "reg_lambda": 9.208866188223618},
        7924: {"n_estimators": 1604, "max_depth": 8, "learning_rate": 0.2911058056208414, "subsample": 0.6119067139645591, "colsample_bytree": 0.5021203624491055, "min_child_weight": 10, "reg_alpha": 6.611089229649847, "reg_lambda": 3.137269959121449},
    },
    "2순": {
        9377: {"n_estimators": 1373, "max_depth": 18, "learning_rate": 0.14375283737296865, "subsample": 0.7980945406440833, "colsample_bytree": 0.6991346660937391, "min_child_weight": 9, "reg_alpha": 4.113544158248988, "reg_lambda": 8.722449676167146},
        9555: {"n_estimators": 1238, "max_depth": 10, "learning_rate": 0.21130610113797096, "subsample": 0.9724883267073136, "colsample_bytree": 0.7461372896252737, "min_child_weight": 9, "reg_alpha": 1.2613115437334719, "reg_lambda": 4.626540684661389},
        3949: {"n_estimators": 555, "max_depth": 5, "learning_rate": 0.010570450457952268, "subsample": 0.8154770223216381, "colsample_bytree": 0.8656317002925095, "min_child_weight": 8, "reg_alpha": 6.3239592064058785, "reg_lambda": 7.253226493539258},
        7924: {"n_estimators": 376, "max_depth": 8, "learning_rate": 0.010135233560877567, "subsample": 0.8403049885933405, "colsample_bytree": 0.9086073837242736, "min_child_weight": 10, "reg_alpha": 5.257413825544417, "reg_lambda": 8.174033051590143},
    },
    "3순": {
        9377: {"n_estimators": 357, "max_depth": 20, "learning_rate": 0.18816763384815463, "subsample": 0.9990202925914138, "colsample_bytree": 0.9845285571832534, "min_child_weight": 1, "reg_alpha": 5.739501300085931, "reg_lambda": 3.635319130826896},
        9555: {"n_estimators": 2076, "max_depth": 17, "learning_rate": 0.24476220280869376, "subsample": 0.5237068024969845, "colsample_bytree": 0.913814549682912, "min_child_weight": 3, "reg_alpha": 1.713088294555641, "reg_lambda": 1.7002498064252087},
        3949: {"n_estimators": 2977, "max_depth": 7, "learning_rate": 0.12408611365695867, "subsample": 0.9667355329470387, "colsample_bytree": 0.6033825686451063, "min_child_weight": 1, "reg_alpha": 8.827814800067877, "reg_lambda": 1.1692290472423528},
        7924: {"n_estimators": 2216, "max_depth": 7, "learning_rate": 0.18676439097855924, "subsample": 0.799154585126457, "colsample_bytree": 0.7693793019413694, "min_child_weight": 3, "reg_alpha": 8.562157011429123, "reg_lambda": 0.6944695239816789},
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
        best_model.fit(X, y)

        model_save_path = f'사과_XGB_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '사과', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '사과', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '사과', '3순'] = y_preds['y_pred_3']


### 상추

In [34]:
train = pd.read_csv("train_상추.csv", encoding = 'cp949')
test = pd.read_csv("test_상추.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

y_preds = {}

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 1312, "max_depth": 15, "learning_rate": 0.11895775817463827, "subsample": 0.8153025722340418, "colsample_bytree": 0.8720055423352995, "min_child_weight": 1, "reg_alpha": 0.909983232747657, "reg_lambda": 7.860944363938349},
        9555: {"n_estimators": 2500, "max_depth": 16, "learning_rate": 0.11232146582832961, "subsample": 0.8141884080927602, "colsample_bytree": 0.6604646126667512, "min_child_weight": 2, "reg_alpha": 4.4474470815374225, "reg_lambda": 8.89722400498199},
        3949: {"n_estimators": 1852, "max_depth": 14, "learning_rate": 0.17197036829361365, "subsample": 0.8422320412303756, "colsample_bytree": 0.9110552688768728, "min_child_weight": 4, "reg_alpha": 5.948966664786619, "reg_lambda": 5.928163872292289},
        7924: {"n_estimators": 1585, "max_depth": 6, "learning_rate": 0.11870060285522362, "subsample": 0.8682700229817323, "colsample_bytree": 0.7828100242402877, "min_child_weight": 2, "reg_alpha": 9.079618633665191, "reg_lambda": 1.6824945246105618},
    },
    "2순": {
        9377: {"n_estimators": 1874, "max_depth": 13, "learning_rate": 0.17305613970672298, "subsample": 0.9501698793451449, "colsample_bytree": 0.5639420366251624, "min_child_weight": 1, "reg_alpha": 6.829047419703269, "reg_lambda": 8.401325822693835},
        9555: {"n_estimators": 2742, "max_depth": 18, "learning_rate": 0.08463924644250331, "subsample": 0.9110914957913919, "colsample_bytree": 0.5321303203611113, "min_child_weight": 2, "reg_alpha": 4.094156660791642, "reg_lambda": 4.305522417585725},
        3949: {"n_estimators": 984, "max_depth": 7, "learning_rate": 0.06906420759422413, "subsample": 0.7758886181633363, "colsample_bytree": 0.7524316083364925, "min_child_weight": 2, "reg_alpha": 4.836010397548115, "reg_lambda": 0.4547142340554328},
        7924: {"n_estimators": 747, "max_depth": 17, "learning_rate": 0.1956783154690147, "subsample": 0.6663109957391311, "colsample_bytree": 0.6357924685325047, "min_child_weight": 2, "reg_alpha": 8.267181183775373, "reg_lambda": 0.7913174953785275},
    },
    "3순": {
        9377: {"n_estimators": 1438, "max_depth": 3, "learning_rate": 0.15359459075313742, "subsample": 0.6088636011039084, "colsample_bytree": 0.5595309612590585, "min_child_weight": 10, "reg_alpha": 6.824202747452386, "reg_lambda": 8.753034790783037},
        9555: {"n_estimators": 1314, "max_depth": 20, "learning_rate": 0.2354621856771178, "subsample": 0.8490882670860475, "colsample_bytree": 0.7757478782446058, "min_child_weight": 3, "reg_alpha": 3.47105855818328, "reg_lambda": 0.13707376973527374},
        3949: {"n_estimators": 2992, "max_depth": 15, "learning_rate": 0.20307534109597353, "subsample": 0.9992515707579245, "colsample_bytree": 0.5582950458929625, "min_child_weight": 3, "reg_alpha": 0.39019290927353123, "reg_lambda": 1.2808854960469607},
        7924: {"n_estimators": 1576, "max_depth": 9, "learning_rate": 0.17536961390308486, "subsample": 0.867634658488321, "colsample_bytree": 0.5344301302420074, "min_child_weight": 1, "reg_alpha": 0.023755462725737697, "reg_lambda": 3.639499643670708},
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
        best_model.fit(X, y)

        model_save_path = f'상추_XGB_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '상추', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '상추', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '상추', '3순'] = y_preds['y_pred_3']


### 양파

In [35]:
train = pd.read_csv("train_양파.csv", encoding = 'cp949')
test = pd.read_csv("test_양파.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

y_preds = {}

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 2711, "max_depth": 11, "learning_rate": 0.0822572890904799, "subsample": 0.9830148317408216, "colsample_bytree": 0.5860457493813139, "min_child_weight": 3, "reg_alpha": 6.430164396238038, "reg_lambda": 1.4802328034212109},
        9555: {"n_estimators": 2285, "max_depth": 10, "learning_rate": 0.11466159894556924, "subsample": 0.8950035329739154, "colsample_bytree": 0.7109184779214015, "min_child_weight": 4, "reg_alpha": 1.681903689270925, "reg_lambda": 0.9547416223204441},
        3949: {"n_estimators": 1966, "max_depth": 7, "learning_rate": 0.14127266585706388, "subsample": 0.5038226371862348, "colsample_bytree": 0.7818094378865361, "min_child_weight": 10, "reg_alpha": 3.611236597565949, "reg_lambda": 5.368293925935806},
        7924: {"n_estimators": 2564, "max_depth": 11, "learning_rate": 0.055563700644852984, "subsample": 0.5002180419430756, "colsample_bytree": 0.6864220977212427, "min_child_weight": 10, "reg_alpha": 8.580356500089822, "reg_lambda": 3.7435079585061706}
    },
    "2순": {
        9377: {"n_estimators": 1744, "max_depth": 17, "learning_rate": 0.2506407481132115, "subsample": 0.9585939084319821, "colsample_bytree": 0.5651261171352302, "min_child_weight": 6, "reg_alpha": 6.1782568817375125, "reg_lambda": 4.725772067330972},
        9555: {"n_estimators": 1434, "max_depth": 7, "learning_rate": 0.21441134366350736, "subsample": 0.8817436775976175, "colsample_bytree": 0.9222369356588789, "min_child_weight": 8, "reg_alpha": 4.273523657891209, "reg_lambda": 4.236383552993547},
        3949: {"n_estimators": 1156, "max_depth": 13, "learning_rate": 0.2754000178331954, "subsample": 0.9580044648243046, "colsample_bytree": 0.7095871826559714, "min_child_weight": 1, "reg_alpha": 5.656193120535485, "reg_lambda": 0.06212075959845681},
        7924: {"n_estimators": 1211, "max_depth": 3, "learning_rate": 0.06821836290440021, "subsample": 0.5016643880174569, "colsample_bytree": 0.7213983541808716, "min_child_weight": 10, "reg_alpha": 0.8573673033233379, "reg_lambda": 4.178394376510033}
    },
    "3순": {
        9377: {"n_estimators": 632, "max_depth": 16, "learning_rate": 0.24527920738212627, "subsample": 0.8578801100943771, "colsample_bytree": 0.6104297107893758, "min_child_weight": 8, "reg_alpha": 1.010607423848899, "reg_lambda": 9.06142099891432},
        9555: {"n_estimators": 703, "max_depth": 9, "learning_rate": 0.29861583250631996, "subsample": 0.871038949736396, "colsample_bytree": 0.5727369290704256, "min_child_weight": 6, "reg_alpha": 8.592453638427925, "reg_lambda": 2.0717976820380186},
        3949: {"n_estimators": 2115, "max_depth": 8, "learning_rate": 0.2800091580088896, "subsample": 0.7431088904230398, "colsample_bytree": 0.845305638710943, "min_child_weight": 1, "reg_alpha": 9.233760477290605, "reg_lambda": 1.3514933732515129},
        7924: {"n_estimators": 534, "max_depth": 19, "learning_rate": 0.19122541333982607, "subsample": 0.9074922138072681, "colsample_bytree": 0.6474081415991231, "min_child_weight": 6, "reg_alpha": 5.369472403273452, "reg_lambda": 4.708413369461076}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
        best_model.fit(X, y)

        model_save_path = f'양파_XGB_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '양파', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '양파', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '양파', '3순'] = y_preds['y_pred_3']


### 배

In [36]:
train = pd.read_csv("train_배.csv", encoding = 'cp949')
test = pd.read_csv("test_배.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

y_preds = {}

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 2679, "max_depth": 12, "learning_rate": 0.08465909430485449, "subsample": 0.776021419389621, "colsample_bytree": 0.5782041897718113, "min_child_weight": 2, "reg_alpha": 9.706563245861238, "reg_lambda": 6.170240281388082},
        9555: {"n_estimators": 2002, "max_depth": 3, "learning_rate": 0.020071354049952087, "subsample": 0.5311763262646161, "colsample_bytree": 0.6761694179761513, "min_child_weight": 1, "reg_alpha": 6.362402626397053, "reg_lambda": 6.97041080867747},
        3949: {"n_estimators": 343, "max_depth": 18, "learning_rate": 0.0858316448557873, "subsample": 0.8665421790607211, "colsample_bytree": 0.5474177156356084, "min_child_weight": 4, "reg_alpha": 5.904984075875019, "reg_lambda": 9.925913001189544},
        7924: {"n_estimators": 2599, "max_depth": 14, "learning_rate": 0.09889075189717655, "subsample": 0.7972896851161305, "colsample_bytree": 0.5110205241793789, "min_child_weight": 3, "reg_alpha": 6.280139608045091, "reg_lambda": 4.139535385256175}
    },
    "2순": {
        9377: {"n_estimators": 1049, "max_depth": 16, "learning_rate": 0.16608864900088965, "subsample": 0.9197352573058059, "colsample_bytree": 0.842569712465772, "min_child_weight": 3, "reg_alpha": 9.235569825779569, "reg_lambda": 2.0056925321928034},
        9555: {"n_estimators": 917, "max_depth": 3, "learning_rate": 0.0283946853492736, "subsample": 0.9660865543723807, "colsample_bytree": 0.6563105445729674, "min_child_weight": 4, "reg_alpha": 7.676282424067879, "reg_lambda": 5.188613818151924},
        3949: {"n_estimators": 809, "max_depth": 8, "learning_rate": 0.13317265788368643, "subsample": 0.7858770858826861, "colsample_bytree": 0.7842747517765137, "min_child_weight": 3, "reg_alpha": 7.882833968935417, "reg_lambda": 1.167492053679293},
        7924: {"n_estimators": 1968, "max_depth": 10, "learning_rate": 0.25657065169504106, "subsample": 0.9134193035846296, "colsample_bytree": 0.7753144457380668, "min_child_weight": 5, "reg_alpha": 7.320891131826887, "reg_lambda": 2.64024094037875}
    },
    "3순": {
        9377: {"n_estimators": 1590, "max_depth": 12, "learning_rate": 0.22027085564265703, "subsample": 0.7645469026319026, "colsample_bytree": 0.9884977733667253, "min_child_weight": 9, "reg_alpha": 1.8572576978655824, "reg_lambda": 2.0629572162293672},
        9555: {"n_estimators": 1277, "max_depth": 5, "learning_rate": 0.0953661346925796, "subsample": 0.5050645344359352, "colsample_bytree": 0.6228038238195374, "min_child_weight": 8, "reg_alpha": 2.096918453438559, "reg_lambda": 7.530826990956779},
        3949: {"n_estimators": 1560, "max_depth": 8, "learning_rate": 0.28411645711224076, "subsample": 0.913274578936212, "colsample_bytree": 0.9753763431105147, "min_child_weight": 10, "reg_alpha": 3.8907828045042527, "reg_lambda": 9.333776804057237},
        7924: {"n_estimators": 1520, "max_depth": 12, "learning_rate": 0.09733980082279606, "subsample": 0.9936446543747804, "colsample_bytree": 0.8643926326307549, "min_child_weight": 6, "reg_alpha": 5.9639116051550385, "reg_lambda": 8.967936282316014}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순' 

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
        best_model.fit(X, y)

        model_save_path = f'배_XGB_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '배', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '배', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '배', '3순'] = y_preds['y_pred_3']


# Final Sub

In [37]:
submission_path = 'sample_submission.csv'
final_test_data = sub_test.copy()
submission_data = pd.read_csv(submission_path, encoding='utf-8')
test_columns = ['시점', '품목명', '1순', '2순', '3순']
final_test_filtered = final_test_data[test_columns]
pivot_test_data = final_test_filtered.pivot(index='시점', columns='품목명', values=['1순', '2순', '3순'])
for step in ['1순', '2순', '3순']:
    for product in submission_data.columns[1:]: 
        col_name = f'{step}'
        submission_data.loc[submission_data['시점'].str.contains(step, regex=False), product] = pivot_test_data[col_name, product].values
submission_data.to_csv('XGB.csv', index=False, encoding='utf-8')
submission_data

,시점,감자,건고추,깐마늘(국산),대파,무,배추,사과,상추,양파,배
0,TEST_00+1순,38124.093750,687556.1875,169100.062500,2305.347412,19352.425781,11393.666016,25276.578125,1187.257568,1275.109863,34433.347656
1,TEST_00+2순,39384.132812,714900.3750,169101.843750,2461.057617,13369.540039,7528.818359,26164.234375,1185.140381,1172.354370,33928.492188
2,TEST_00+3순,39460.667969,666969.0000,168872.515625,2476.607422,12056.388672,6923.612305,25059.361328,1177.918579,1032.725098,33387.062500
3,TEST_01+1순,38919.625000,658799.9375,169100.062500,1888.259155,12278.299805,6279.210938,23281.472656,1004.476013,1400.924561,31843.650391
4,TEST_01+2순,40833.187500,641851.1875,169101.843750,1673.533325,12647.471680,6209.206055,23833.597656,1032.776489,1238.763550,34861.437500
...,...,...,...,...,...,...,...,...,...,...,...
70,TEST_23+2순,42399.332031,658909.9375,169101.843750,1595.145630,9769.289062,4100.420898,23300.863281,963.473206,1220.163818,32870.277344
71,TEST_23+3순,42076.203125,630224.1875,168872.515625,1400.934448,11442.001953,3721.550293,23981.726562,1036.344971,1066.103027,32395.539062
72,TEST_24+1순,48885.898438,544052.2500,169100.062500,1498.171387,14520.036133,8123.449707,28800.636719,1009.129211,527.100708,38544.285156
73,TEST_24+2순,38061.968750,548834.0625,169101.843750,1297.417725,13961.857422,8330.664062,29420.134766,1122.527588,529.395325,40734.695312
